In [ ]:
# https://github.com/pytorch/pytorch/blob/3cbe7a53a9a1cea2ef2a042f1ab6f7758f7e4d74/torch/csrc/api/include/torch/nn/functional/activation.h#L643

In [ ]:
# https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# https://youtu.be/kCc8FmEb1nY?si=AEN-_b8nxN1nxvDf&t=5248

In [ ]:
# https://github.com/karpathy/nanoGPT/issues/399

In [ ]:
# sentencepiece
# tiktoken

In [ ]:
## deprecated

# @kfp.dsl.component(base_image=PIPELINE_IMAGE_URI) # TODO: replace with lightweight base_image
# def get_configs(bucket_name: str, path_model_config: str, path_tiktoken_config: str) -> dict:
#     import pickle
    
#     from google.cloud import storage


#     storage_client = storage.Client()
#     bucket = storage_client.get_bucket(bucket_name)
    
#     blob_model_config = bucket.blob(path_model_config)
#     blob_tiktoken_config = bucket.blob(path_tiktoken_config)
    
#     with blob_model_config.open(mode='rb') as f:
#         model_config = pickle.load(f)
#     with blob_tiktoken_config.open(mode='rb') as f:
#         tiktoken_config = pickle.load(f)

#     assert model_config['embed_dim'] % model_config['num_heads'] == 0

#     return {'model_config': model_config, 'tiktoken_config': tiktoken_config}


# @kfp.dsl.component(base_image=PIPELINE_IMAGE_URI)
# def get_data(bucket_name: str, data_path: str) -> str:
#     from google.cloud import storage


#     bucket = storage_client.get_bucket(bucket_name)
#     blob = bucket.blob(storage)

#     return blob_data.download_as_text()

# https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts

# @kfp.dsl.container_component
# def prep_deployment(artifact_model: Input[Model], artifact_model_mar: Output[Model]):
#     # Note: python code glitchy in this container_component
#     return kfp.dsl.ContainerSpec(
#         image=PIPELINE_IMAGE_URI,
#         command=[
#             'sh', '-c', '''
#             echo $1\
#             && echo $2\
#             && mkdir -p $1\
#             && echo hello world\ 
#             && torch-model-archiver --model-name model --version 0.1 --serialized-file $2 --handler deployment_handler.py --export-path $1
#             '''
#         ],
#         args=[str(artifact_model_mar.path), str(artifact_model.path) + '/model.pth']
#     )

# @kfp.dsl.component(base_image=PIPELINE_IMAGE_URI)
# def prep_deployment(artifact_model: Input[Model], artifact_model_mar: Output[Model]):
#     import subprocess
#     import os
#     os.makedirs(artifact_model_mar.path, exist_ok=True)
#     cmd = f"torch-model-archiver\
#         --model-name model\
#         --version 0.1\
#         --serialized-file {str(artifact_model.path) + '/model.pth'}\
#         --handler deployment_handler.py\
#         --export-path {str(artifact_model_mar.path)}"
#     subprocess.run([cmd], shell=True)

In [ ]:
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# enc.encode('hi my name is linsu!')
# enc.decode([5303, 616, 1438, 318, 300, 1040, 84, 0])

![](diagram.png)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
B, T, E = 4, 8, 2
x = torch.randn(B, T, E)
xbow = torch.zeros((B, T, E))

In [ ]:
# method 1
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, D)
        xbow[b, t] = torch.mean(xprev, 0) # (D,)

In [ ]:
# method 2
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (-, T, T) @ (B, T, C) = (B, T, C)

In [ ]:
# method 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # (-, T, T) @ (B, T, C) = (B, T, C)

In [ ]:
# method 4: Attention Mechanism
head_dim = 16
query = nn.Linear(E, head_dim, bias=False)
key = nn.Linear(E, head_dim, bias=False)
value = nn.Linear(E, head_dim, bias=False)
q = query(x) # q = Wx (B, T, 16)
k = key(x) # k = Wx (B, T, 16)
v = value(x) # v = Wx (B, T, 16)

# dot product between two vectors measures similarity
# this matrix multiplication dots each vector to every other vector
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (8, 16, T) = (B, T, T)
wei = wei * head_dim**(-0.5)

tril = torch.tril(torch.ones(T, T)) # optional (for decoder)
wei = wei.masked_fill(tril == 0, float('-inf')) # optional (for decoder)
wei = F.softmax(wei, dim=-1)
out = wei @ v # (B, T, T) @ (B, T, H) = (B, T, H)